## Working on pulling edge statistics from images

#### Resources and ideas
* https://akshaysin.github.io/fourier_lpf.html#.XHwzzohKhPb -- using Pass filters to change the output images

### Intial Forier Transform

### Using High Pass Filter

### Using Band Pass Filters

### Pulling stastistics from images